# Notes on resize with antialiasing as Pillow (pytorch interpolate)

In [1]:
import numpy as np
from PIL import Image

i = np.arange(8 * 8 * 3, dtype="uint8").reshape(8, 8, 3)
print(i[0, :, :])

i = Image.fromarray(i)

o = i.resize([2, 8], 2)
print(o.size)
oo = np.asarray(o)
print(oo[0, :, :].tolist())


[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]
 [18 19 20]
 [21 22 23]]
(2, 8)
[[6, 7, 8], [15, 16, 17]]


In [14]:
import numpy as np
from PIL import Image

i = np.arange(8 * 8 * 3, dtype="float").reshape(8, 8, 3)
i = i[:, :, 0]
print(i[0, :])

i = Image.fromarray(i)

o = i.resize([2, 8], 2)
print(o.size)
oo = np.asarray(o)
print(oo[0, :].tolist())


[ 0.  3.  6.  9. 12. 15. 18. 21.]
(2, 8)
[5.6785712242126465, 15.321428298950195]


- Input as RGB stored as uint32 as (r, g, b, 255)

## Compute first line output (size=2)

- let's skip how weights and bounds are computed

Output is computed as following:

$$
output(i) = \sum_{j=0}^{xsize_i} ( input(xmin_i + j) * W(i, j))
$$

```
output_R[o] = R[xmin[o] + 0] * W[0] + R[xmin[o] + 1] * W[1] + ...

mmk = [W[0]_0, W[0]_1, W[1]_0, W[1]_1, W[0]_0, W[0]_1, W[1]_0, W[1]_1, ...]

data_cl = [R0 G0 B0 A0, R1 G1 B1 A1, ...]

data_cf1 = [R00 R01 R02 R03, R04 ...]
data_cf2 = [R10 R11 R12 R13, R14 ...]
data_cf3 = [R20 R21 R22 R13, R14 ...]
data_cf4 = [R30 R31 R32 R13, R14 ...]


pix_cl = [
    R[xmin[o] + 0] 0 R[xmin[o] + 1] 0, G[xmin[o] + 0] 0 G[xmin[o] + 1] 0, ...  
]

pix_cf = [
    R[xmin[o] + 0] 0 R[xmin[o] + 1] 0, R[xmin[o2] + 0] 0 R[xmin[o2] + 1] 0, ...  
]


```

In [ ]:
# uint8_t * __ALIGNED__ data[4] = {src[0], src[1], src[2], 255}
# Vec v = Vec::loadu(data);
# _mm256_loadu_epi32(src[3 * i + offset]);

In [15]:
input_line = [0, 1, 2, 255, 3, 4, 5, 255, 6, 7, 8, 255, 9, 10, 11, 255, 12, 13, 14, 255, 15, 16, 17, 255, 18, 19, 20, 255, 21, 22, 23, 255]

In [16]:
xmin_0, xsize_0 = 0, 6
weights_0 = [0.17857142857142858, 0.25, 0.25, 0.17857142857142858, 0.10714285714285714, 0.035714285714285712, 0.0, 0.0, 0.0]

xmin_1, xsize_1 = 2, 6
weights_1 = [0.035714285714285712, 0.10714285714285714, 0.17857142857142858, 0.25, 0.25, 0.17857142857142858, 0.0, 0.0, 0.0]

In [17]:
output_0 = []

print("- Red: ")
output_r = 0.0
input_r = input_line[::4]  # [0, 3, 6, 9, 12, 15, 18, 21]
for i in range(xsize_0):
    r = input_r[i + xmin_0]
    w = weights_0[i]
    output_r += r * w
    print(r, w, r * w, "->", output_r)
    
output_0.append(int(output_r + 0.5))

print("- Green: ")
output_g = 0.0
input_g = input_line[1::4]  # [0, 3, 6, 9, 12, 15, 18, 21]
for i in range(xsize_0):
    g = input_g[i + xmin_0]
    w = weights_0[i]
    output_g += g * w
    print(g, w, g * w, "->", output_g)
    
output_0.append(int(output_g + 0.5))

print("- Blue: ")
output_b = 0.0
input_b = input_line[2::4]  # [0, 3, 6, 9, 12, 15, 18, 21]
for i in range(xsize_0):
    b = input_b[i + xmin_0]
    w = weights_0[i]
    output_b += b * w
    print(b, w, b * w, "->", output_b)
    
output_0.append(int(output_b + 0.5))
output_0

- Red: 
0 0.17857142857142858 0.0 -> 0.0
3 0.25 0.75 -> 0.75
6 0.25 1.5 -> 2.25
9 0.17857142857142858 1.6071428571428572 -> 3.857142857142857
12 0.10714285714285714 1.2857142857142856 -> 5.142857142857142
15 0.03571428571428571 0.5357142857142857 -> 5.678571428571428
- Green: 
1 0.17857142857142858 0.17857142857142858 -> 0.17857142857142858
4 0.25 1.0 -> 1.1785714285714286
7 0.25 1.75 -> 2.928571428571429
10 0.17857142857142858 1.7857142857142858 -> 4.714285714285714
13 0.10714285714285714 1.3928571428571428 -> 6.107142857142858
16 0.03571428571428571 0.5714285714285714 -> 6.678571428571429
- Blue: 
2 0.17857142857142858 0.35714285714285715 -> 0.35714285714285715
5 0.25 1.25 -> 1.6071428571428572
8 0.25 2.0 -> 3.607142857142857
11 0.17857142857142858 1.9642857142857144 -> 5.571428571428571
14 0.10714285714285714 1.5 -> 7.071428571428571
17 0.03571428571428571 0.6071428571428571 -> 7.678571428571428


[6, 7, 8]

In [18]:
output_1 = []

print("- Red: ")
output_r = 0.0
input_r = input_line[::4]  # [0, 3, 6, 9, 12, 15, 18, 21]
for i in range(xsize_1):
    r = input_r[i + xmin_1]
    w = weights_1[i]
    output_r += r * w
    print(r, w, r * w, "->", output_r)
    
output_1.append(int(output_r + 0.5))

print("- Green: ")
output_g = 0.0
input_g = input_line[1::4]  # [0, 3, 6, 9, 12, 15, 18, 21]
for i in range(xsize_1):
    g = input_g[i + xmin_1]
    w = weights_1[i]
    output_g += g * w
    print(g, w, g * w, "->", output_g)
    
output_1.append(int(output_g + 0.5))

print("- Blue: ")
output_b = 0.0
input_b = input_line[2::4]  # [0, 3, 6, 9, 12, 15, 18, 21]
for i in range(xsize_1):
    b = input_b[i + xmin_1]
    w = weights_1[i]
    output_b += b * w
    print(b, w, b * w, "->", output_b)
    
output_1.append(int(output_b + 0.5))
output_1

- Red: 
6 0.03571428571428571 0.21428571428571427 -> 0.21428571428571427
9 0.10714285714285714 0.9642857142857142 -> 1.1785714285714284
12 0.17857142857142858 2.142857142857143 -> 3.321428571428571
15 0.25 3.75 -> 7.071428571428571
18 0.25 4.5 -> 11.571428571428571
21 0.17857142857142858 3.75 -> 15.321428571428571
- Green: 
7 0.03571428571428571 0.25 -> 0.25
10 0.10714285714285714 1.0714285714285714 -> 1.3214285714285714
13 0.17857142857142858 2.3214285714285716 -> 3.6428571428571432
16 0.25 4.0 -> 7.642857142857143
19 0.25 4.75 -> 12.392857142857142
22 0.17857142857142858 3.928571428571429 -> 16.32142857142857
- Blue: 
8 0.03571428571428571 0.2857142857142857 -> 0.2857142857142857
11 0.10714285714285714 1.1785714285714286 -> 1.4642857142857144
14 0.17857142857142858 2.5 -> 3.9642857142857144
17 0.25 4.25 -> 8.214285714285715
20 0.25 5.0 -> 13.214285714285715
23 0.17857142857142858 4.107142857142858 -> 17.321428571428573


[15, 16, 17]

## Weights normalization

- https://github.com/uploadcare/pillow-simd/blob/668aa48d12305b8f093958792a5e4f690c2583d6/src/libImaging/Resample.c#L275

```
    for (coefs_precision = 0; coefs_precision < PRECISION_BITS; coefs_precision += 1) {
        int next_value = (int) (0.5 + maxkk * (1 << (coefs_precision + 1)));
        // The next value will be outside of the range, so just stop
        if (next_value >= (1 << MAX_COEFS_PRECISION))
            break;
    }
```

In [6]:
maxkk_list = [0.001, 0.01, 0.1, 0.25, 0.5, 0.75, 0.99, 0.999]

In [11]:
c = 14
max_value = 1 << 15
for maxkk in maxkk_list:
    value = int(0.5 + maxkk * (1 << (c + 1)))
    print(maxkk, value, max_value)

0.001 33 32768
0.01 328 32768
0.1 3277 32768
0.25 8192 32768
0.5 16384 32768
0.75 24576 32768
0.99 32440 32768
0.999 32735 32768


In [15]:
c = 14
int(0.5 + (1 << (c + 1))) == 1 << 15

True

In [4]:
a = list(range(8 * 9 * 3))
s = 9 * 3
for i in range(8):
    b = []
    for i, v in enumerate(a[s * i:s * (i + 1)]):
        if i > 0 and i % 3 == 0:
            b.append(255)
            b.append(v)
        else:
            b.append(v)
    b.append(255)
    print(b)

[0, 1, 2, 255, 3, 4, 5, 255, 6, 7, 8, 255, 9, 10, 11, 255, 12, 13, 14, 255, 15, 16, 17, 255, 18, 19, 20, 255, 21, 22, 23, 255, 24, 25, 26, 255]
[27, 28, 29, 255, 30, 31, 32, 255, 33, 34, 35, 255, 36, 37, 38, 255, 39, 40, 41, 255, 42, 43, 44, 255, 45, 46, 47, 255, 48, 49, 50, 255, 51, 52, 53, 255]
[54, 55, 56, 255, 57, 58, 59, 255, 60, 61, 62, 255, 63, 64, 65, 255, 66, 67, 68, 255, 69, 70, 71, 255, 72, 73, 74, 255, 75, 76, 77, 255, 78, 79, 80, 255]
[81, 82, 83, 255, 84, 85, 86, 255, 87, 88, 89, 255, 90, 91, 92, 255, 93, 94, 95, 255, 96, 97, 98, 255, 99, 100, 101, 255, 102, 103, 104, 255, 105, 106, 107, 255]
[108, 109, 110, 255, 111, 112, 113, 255, 114, 115, 116, 255, 117, 118, 119, 255, 120, 121, 122, 255, 123, 124, 125, 255, 126, 127, 128, 255, 129, 130, 131, 255, 132, 133, 134, 255]
[135, 136, 137, 255, 138, 139, 140, 255, 141, 142, 143, 255, 144, 145, 146, 255, 147, 148, 149, 255, 150, 151, 152, 255, 153, 154, 155, 255, 156, 157, 158, 255, 159, 160, 161, 255]
[162, 163, 164, 255, 165

In [8]:
a = list(range(8 * 9 * 3))
s = 9 * 3
for i in range(8):
    b = []
    for i, v in enumerate(a[s * i:s * (i + 1)]):
        b.append(v)
    print(b)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
[27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]
[54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80]
[81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107]
[108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134]
[135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161]
[162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188]
[189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215]


In [10]:
import torch

x = torch.randint(0, 256, size=(1, 3, 32, 32), dtype=torch.uint8)

```
// lineIn0 + stride * (x + xmin) + 16 <= lineIn0 + stride * (xmax + xmin)
// --> x <= xmax - 16.0 / stride --> x < xmax + 1 - 16.0 / stride


// const auto b4_xmax = int(xmax - 16.0 / stride + 1 + 0.5);
```

In [34]:
10 + 1 - 16.0 / 4

7.0

In [37]:
from math import floor

for xmax in range(4, 100):
    for size in [16.0, 8.0]:
        for stride in [3, 4]:
            v0 = xmax + 1 - size / stride
            v1 = xmax + 1 - int(size / stride)
            assert v0 <= v1 and v0 + 1 > v1, (v0, v1)

In [28]:


ceil(1 + 0.0)

1

In [18]:
stride = 4

for xmax in range(4, 200):

    b4_xmax = int(xmax - 16.0 / stride + 1 + 0.5)
    b4_max = int(16.0 / stride + 0.5)
    # b4_xmax, xmax - 3, b4_max, xmax - b4_max
    assert b4_xmax == xmax - b4_max + 1, (xmax, b4_xmax, xmax - b4_max + 1, b4_max)

In [20]:
xmax = 10
stride = 4

b4_xmax = int(xmax - 16.0 / stride + 1 + 0.5)
b4_max = int(16.0 / stride + 0.5)
b4_xmax, xmax - b4_max + 1

(7, 7)

In [22]:
int(16.0 / stride + 0.5)

4

In [16]:
stride = 3

for xmax in range(4, 100):

    b4_xmax = int(xmax - 16.0 / stride + 1 + 0.5)
    b4_max = int(16.0 / stride + 0.5)
    # b4_xmax, xmax - 3, b4_max, xmax - b4_max
    assert b4_xmax == xmax - b4_max + 1, (xmax, b4_xmax, xmax - b4_max + 1, b4_max)